In [1]:
from global_vars import *
import torch
from transformers import AutoModelWithLMHead, AutoTokenizer, T5ForConditionalGeneration
from datasets import Dataset, load_from_disk
from torch.utils.data import DataLoader, RandomSampler, BatchSampler
from tqdm import tqdm
import pandas as pd
import wandb
import pickle
import sys
from datetime import datetime

dict = {
    "model_name": model_t5,
    "learning_rate": learning_rate,
    "max_length_input": max_length_input,
    "doTruncate": doTruncate,
    "doPadding": doPadding,
    "split_train_test": split_train_test,
    "split_train_val": split_train_val,
    "batch_size": batch_size,
    "epochs": epochs,
    "layers_freeze": layers_freeze,
}
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

model_saved = "t52023-03-18-13-13-00.pt"
tokenizer = AutoTokenizer.from_pretrained(model_t5)
model = T5ForConditionalGeneration.from_pretrained(model_t5).to(device)
model.load_state_dict(torch.load(model_saved, map_location=torch.device('cpu')))

/mnt/SSD3/dud/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mnt/SSD3/dud/miniconda3/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your pref

<All keys matched successfully>

In [2]:
def compute_accuracy(preds, labels):
    o2 = [tokenizer.decode(preds[i], skip_special_tokens=True) for i in range(preds.shape[0])]
    b3 = [tokenizer.decode(labels[i], skip_special_tokens=True) for i in range(labels.shape[0])]
    count = 0
    for i in range(len(o2)):
        if o2[i].lower().strip() == b3[i].lower().strip():
            count += 1
    return count

In [3]:
time = "2023-03-18-04-46-28"
testing_dataset_input = load_from_disk("dataset" + time + "/testing_dataset_input")
testing_dataset_labels = load_from_disk("dataset" + time + "/testing_dataset_labels")

In [4]:
testing_dataset = Dataset.from_dict({"input_ids": testing_dataset_input['input_ids'], #train input ids
                                "attention_mask": testing_dataset_input['attention_mask'],
                             "labels_input_ids": testing_dataset_labels['input_ids'] #label input ids
                             }).with_format("torch")
test_dataloader = DataLoader(testing_dataset, batch_size=batch_size, drop_last=False)

In [5]:
val_acc = 0
for batch in tqdm(test_dataloader):
    output = model(input_ids=batch["input_ids"].to(device),
                    attention_mask=batch["attention_mask"].to(device),
                     labels=batch["labels_input_ids"].to(device))
    outs2 = model.generate(batch["input_ids"].to(device), max_new_tokens=4).to(device)
    val_acc += compute_accuracy(outs2, batch["labels_input_ids"])

print(val_acc / len(testing_dataset))

100%|██████████| 663/663 [06:19<00:00,  1.75it/s]

0.8108974358974359
